In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

### Delta Hedging

In [ ]:
def delta_hedge_long(S0, K, T, r, vol, N, M, opttype="C"):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    hedge_position_avg=np.zeros(N+1)
    cash_position_avg=np.zeros(N+1)
    PnL_avg = np.zeros(N + 1)
    time=np.linspace(0, T, N+1)

    for i in range (M):
        St=S0
        hedge_position=np.zeros(N+1)
        cash_position=np.zeros(N+1)
        PnL = np.zeros(N + 1)

        if opttype=="C":
            delta=opt.delta_calc(r, St, K, T, vol, type="c")
        else:
            delta=opt.delta_calc(r, St, K, T, vol, type="p")

        if opttype=="C":
            option_init_value=opt.black_scholes(r, St, K, T, vol, type="c")
        else:
            option_init_value=opt.black_scholes(r, St, K, T, vol, type="p")

        hedge_position[0]=delta
        PnL[0] = hedge_position[0]*St - option_init_value

        for j in range (1, N+1):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn

            T_remaining = T - time[j]

            if opttype=="C":
                new_delta=opt.delta_calc(r, St, K, T_remaining, vol, type="c")
            else:
                new_delta=opt.delta_calc(r, St, K, T_remaining, vol, type="p")

            if opttype=="C":
                option_value=opt.black_scholes(r, St, K, T_remaining, vol, type="c")
            else:
                option_value=opt.black_scholes(r, St, K, T_remaining, vol, type="p")
            
            delta_change=new_delta-hedge_position[j-1]

            hedge_position[j]=new_delta
            if opttype=="C":
                cash_position[j]=cash_position[j-1]+delta_change*St
            else:
                cash_position[j]=cash_position[j-1]-delta_change*St

            PnL[j] = hedge_position[j]*St + cash_position[j] -option_value
        
        hedge_position_avg += hedge_position
        cash_position_avg += cash_position
        PnL_avg += PnL

    hedge_position_avg /= M
    cash_position_avg /= M
    PnL_avg /= M

    return hedge_position_avg, cash_position_avg, PnL_avg, time

### Gamma hedging

In [ ]:
def gamma_hedging_long(S0, K, T, r, vol, c, N, M, opttype="C"):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    hedged_position_avg = np.zeros(N+1)
    hedged_position_gamma_avg = np.zeros(N+1)
    cash_position_avg = np.zeros(N+1)
    PnL_avg = np.zeros(N + 1)
    time = np.linspace(0, T, N+1)

    for i in range(M):
        St = S0
        hedged_position = np.zeros(N+1)
        hedged_position_gamma = np.zeros(N+1)
        cash_position = np.zeros(N+1)
        PnL = np.zeros(N+1)

        if opttype == "C":
            delta = opt.delta_calc(r, St, K, T, vol, type="c")
            gamma = opt.gamma_calc(r, St, K, T, vol, type="c")
        else:
            delta = opt.delta_calc(r, St, K, T, vol, type="p")
            gamma = opt.gamma_calc(r, St, K, T, vol, type="p")

        option_init_value = opt.black_scholes(r, St, K, T, vol, type=opttype.lower())

        ### Initial Hedge
        hedged_position[0] = delta * St
        if opttype == "C":
            second_option = opt.black_scholes(r, St, K, c*T, vol, type="c")
            second_gamma = opt.gamma_calc(r, St, K, c*T, vol, type="c")
        else:
            second_option = opt.black_scholes(r, St, K, c*T, vol, type="p")
            second_gamma = opt.gamma_calc(r, St, K, c*T, vol, type="p")

        ### Gamma Hedge
        hedged_position_gamma[0] = (gamma/second_gamma) * second_option

        PnL[0] = hedged_position[0] + hedged_position_gamma[0] - option_init_value

        for j in range(1, N+1):
            epsilon = np.random.normal()
            Stn = St * np.exp(nudt + volsdt * epsilon)
            St = Stn
            T_remaining = T - time[j]

            if opttype == "C":
                new_delta = opt.delta_calc(r, St, K, T_remaining, vol, type="c")
                new_gamma = opt.gamma_calc(r, St, K, T_remaining, vol, type="c")
                option_value = opt.black_scholes(r, St, K, T_remaining, vol, type="c")
            else:
                new_delta = opt.delta_calc(r, St, K, T_remaining, vol, type="p")
                new_gamma = opt.gamma_calc(r, St, K, T_remaining, vol, type="p")
                option_value = opt.black_scholes(r, St, K, T_remaining, vol, type="p")

            ### Delta Hedge Update
            delta_change = new_delta - hedged_position[j-1]
            hedged_position[j] = new_delta * St

            ### Gamma Hedge Update
            if opttype == "C":
                new_second_option = opt.black_scholes(r, St, K, c*T_remaining, vol, type="c")
                new_second_gamma = opt.gamma_calc(r, St, K, c*T_remaining, vol, type="c")
            else:
                new_second_option = opt.black_scholes(r, St, K, c*T_remaining, vol, type="p")
                new_second_gamma = opt.gamma_calc(r, St, K, c*T_remaining, vol, type="p")

            gamma_change = new_gamma - hedged_position_gamma[j-1]
            hedged_position_gamma[j] = (new_gamma / new_second_gamma) * new_second_option

            ### Update Cash Position
            cash_position[j] = cash_position[j-1] + delta_change * St + gamma_change * new_second_option
            cash_position[j] *= np.exp(r * dt)

            ### PnL Calculation
            PnL[j] = hedged_position[j] + hedged_position_gamma[j] + cash_position[j] - option_value

        hedged_position_avg += hedged_position
        hedged_position_gamma_avg += hedged_position_gamma
        cash_position_avg += cash_position
        PnL_avg += PnL

    hedged_position_avg /= M
    hedged_position_gamma_avg /= M
    cash_position_avg /= M
    PnL_avg /= M

    return hedged_position_avg, hedged_position_gamma_avg, cash_position_avg, PnL_avg, time